In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/data_processing/')
import data_treatment as dt

In [66]:

##Get train and test dataset for pradictions
def get_train_test(dataset, portofolio_id):
    portfolio = portfolio1[portfolio1['cliente_flag'] == portofolio_id]
    market = portfolio1[portfolio1['cliente_flag'] == 0]

    market_sample = market.sample(portfolio.shape[0])

    portfolio_num = portfolio._get_numeric_data()
    market_num =  market_sample._get_numeric_data()

    portfolio_train = pd.concat([portfolio_num, market_num], sort = False)

    portfolio_train = portfolio_train.fillna(0)
    market_test = market.fillna(0)

    return(portfolio_train, market_test)


def get_importance(dataset):
    logreg = LogisticRegression()

    rfe = RFE(logreg, 20)
    rfe = rfe.fit(dataset.drop('cliente_flag', axis=1), dataset['cliente_flag'])

    rfe_support = rfe.support_
    print(rfe.ranking_)

    feat_importance = []
    col_list = dataset.drop('cliente_flag', axis=1).columns.tolist()

    for i in range(0, len(col_list)):
        if(rfe_support[i]== True):
            feat_importance.append(col_list[i])

    return(feat_importance)

def train(dataset,features):
    x_train, x_test, y_train, y_test = train_test_split(dataset.drop('cliente_flag',axis=1), 
                                                    dataset['cliente_flag'], test_size=0.30, 
                                                    random_state=101, stratify=dataset['cliente_flag'])
    
    logmodel = LogisticRegression()
    logmodel.fit(x_train[features],y_train)
    predictions = logmodel.predict(x_test[features])
    predictions_prob = logmodel.predict_proba(x_test[features])
    
    return(logmodel,predictions,y_test)

def metadata(predictions_train_ytest,predictions_train):
    classification = classification_report(predictions_train_ytest,predictions_train)
    confusion = confusion_matrix(predictions_train_ytest,predictions_train)
    print(classification)
    print(confusion)
    return(classification,confusion)

def predict(input_data,logmodel,features):
    #print("==> PREDICT DATASET {}".format(input_data))
    predictions = logmodel.predict(input_data[features])
    predictions_prob = logmodel.predict_proba(input_data[features])
    return(predictions, predictions_prob)

In [ ]:
portfolio = dt.fetch_market(2)
portfolio_train, market_test = get_train_test(portfolio, 2)
feat_importance = get_importance(portfolio_train)
feat_importance
#feat_importance = get_importance(portfolio_train)

/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/data_processing/data_treatment.py:91: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  col_list_selector = df.columns.str.contains( pattern , regex = regexp)


In [30]:
#feat_importance = get_importance(portfolio_train)
logmodel, predictions_train, predictions_train_ytest = train(portfolio_train,feat_importance)

/home/marcelo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [36]:
print(classification_report(predictions_train_ytest,predictions_train))

              precision    recall  f1-score   support

           0       0.95      0.62      0.75       166
           1       0.72      0.97      0.83       167

   micro avg       0.80      0.80      0.80       333
   macro avg       0.84      0.80      0.79       333
weighted avg       0.84      0.80      0.79       333



In [37]:
print(confusion_matrix(predictions_train_ytest,predictions_train))

[[103  63]
 [  5 162]]


In [49]:
predictions, predictions_prob =  predict(market_test,logmodel,feat_importance)

In [56]:
market_test['predictions'] = predictions
market_test['Prob_N'] = predictions_prob[0:,0]
market_test['Prob_Y'] = predictions_prob[0:,1]

In [63]:
market_test.sort_values('Prob_Y', ascending=[False])[['predictions','Prob_Y','Prob_N']].to_csv('LR_rank_01.csv')

In [65]:
rank01 = market_test.sort_values('Prob_Y', ascending=[False])

In [ ]:
rank02 = market_test.sort_values('Prob_Y', ascending=[False])

In [ ]:
rank03 = market_test.sort_values('Prob_Y', ascending=[False])